In [2]:
from tqdm import tqdm
from transformers import (AutoModelForSequenceClassification, AutoTokenizer, pipeline)

In [5]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from tqdm import tqdm


model = AutoModelForSequenceClassification.from_pretrained("facebook/bart-large-mnli").to("cpu")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-mnli")
classifier = pipeline(
            "zero-shot-classification",
            model=model,
            tokenizer=tokenizer,
            device=-1  # -1 for CPU
        )

In [6]:
print(model)
print(tokenizer)

BartForSequenceClassification(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
   

In [ ]:
    # Define the categories for classification
    categories = [
        "Staff Professionalism",
        "Communication Effectiveness",
        "Appointment Availability",
        "Waiting Time",
        "Facility Cleanliness",
        "Patient Respect",
        "Treatment Quality",
        "Staff Empathy and Compassion",
        "Administrative Efficiency",
        "Reception Staff Interaction",
        "Environment and Ambiance",
        "Follow-up and Continuity of Care",
        "Accessibility and Convenience",
        "Patient Education and Information",
        "Feedback and Complaints Handling",
        "Test Results",
        "Surgery Website",
        "Telehealth",
        "Vaccinations",
        "Prescriptions and Medication Management",
        "Mental Health Support",
    ]

    # Initialize the list to store labels
    feedback_labels = [""] * len(data)  # Pre-fill with empty strings

    # Process batches
    total_batches = (len(data) + batch_size - 1) // batch_size  # Calculate total number of batches
    for batch, start_index in tqdm(batch_generator(data, "free_text", batch_size), total=total_batches, desc="Processing batches"):
        # Validate and filter batch data
        valid_sentences = [
            (sentence.strip(), idx)
            for idx, sentence in enumerate(batch)
            if isinstance(sentence, str) and sentence.strip()
        ]
        if not valid_sentences:
            continue  # Skip if no valid sentences are present

        sentences, valid_indices = zip(*valid_sentences) if valid_sentences else ([], [])

        try:
            # Perform classification
            model_outputs = classifier(list(sentences), categories)
            # Assign the most relevant category label
            for output, idx in zip(model_outputs, valid_indices):
                feedback_labels[start_index + idx] = output["labels"][0]
        except Exception as e:
            print(f"Error during classification: {e}")
            # Optionally, handle specific actions for failed classification, such as logging or retrying

    # Assign the computed labels back to the data
    data["feedback_labels"] = feedback_labels
    return data